In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy import stats

In [12]:
seeds = pd.read_csv("C:/Users/marco/Desktop/seeds_dataset.txt", sep="\t")

y=seeds.Wheat
y=np.array(y)
X=seeds.loc[:,seeds.columns!="Wheat"]
normalizar = StandardScaler()
X_norm = normalizar.fit_transform(X)
print(X_norm)

[[ 1.42097769e-01  2.15462437e-01  6.06017918e-05 ...  1.41701823e-01
  -9.86151745e-01 -3.83577423e-01]
 [ 1.11880257e-02  8.22375713e-03  4.28515270e-01 ...  1.97432229e-01
  -1.78816620e+00 -9.22013487e-01]
 [-1.92066576e-01 -3.60200562e-01  1.44238325e+00 ...  2.08047544e-01
  -6.67479334e-01 -1.18919199e+00]
 ...
 [-5.67570840e-01 -6.90247348e-01  7.33948301e-01 ... -7.06044846e-02
   3.07658816e+00 -7.18060432e-01]
 [-1.03608992e+00 -1.03564515e+00 -8.01701104e-01 ... -1.12152071e+00
  -6.81351965e-02 -7.42534799e-01]
 [-8.77620233e-01 -9.35863561e-01 -1.10234659e-01 ... -7.55292327e-01
   1.29122264e+00 -7.03783718e-01]]


In [21]:
# Creación de los folds

K=10
cv_strat = StratifiedKFold(n_splits=K,shuffle=True)

max_clusters= 50
min_clusters= np.unique(y).shape[0]
k = []
for i in range(min_clusters, max_clusters+1):
    k.append(str(i))


In [22]:
tabla = pd.DataFrame(columns=k, data=np.zeros((K,max_clusters-min_clusters+1)),dtype=float)
tabla.columns.name='Clusters'
tabla.index.name='CV'

for c in k:
    kmedias = KMeans(n_clusters=int(c), random_state=0)
    cv=0
    for train_index, test_index in cv_strat.split(X_norm,y):
        X_train = X_norm[train_index]
        y_train = y[train_index]
        X_test = X_norm[test_index]
        y_test = y[test_index]
        kmedias.fit(X_train)
        moda=-1 * np.ones(int(c),dtype='int')

        for i in range(int(c)):
            moda[i] = stats.mode(y_train[np.argwhere(kmedias.labels_==i)])[0].astype(int)
        y_test_predict = moda[kmedias.predict(X_test)]
        tabla.loc[cv,c] = accuracy_score(y_test, y_test_predict)
        cv +=1

In [23]:
tabla.mean()

Clusters
3     0.909524
4     0.900000
5     0.885714
6     0.880952
7     0.890476
8     0.895238
9     0.895238
10    0.904762
11    0.900000
12    0.914286
13    0.904762
14    0.909524
15    0.923810
16    0.876190
17    0.890476
18    0.885714
19    0.909524
20    0.880952
21    0.900000
22    0.890476
23    0.923810
24    0.895238
25    0.914286
26    0.890476
27    0.861905
28    0.919048
29    0.904762
30    0.900000
31    0.885714
32    0.909524
33    0.904762
34    0.919048
35    0.890476
36    0.885714
37    0.885714
38    0.909524
39    0.909524
40    0.895238
41    0.876190
42    0.914286
43    0.904762
44    0.919048
45    0.909524
46    0.933333
47    0.895238
48    0.900000
49    0.900000
50    0.895238
dtype: float64

In [24]:
print("el k optimo es: ", tabla.columns[tabla.mean(axis=0).argmax()], " con una tasa de aciertos: ", tabla.mean(axis=0).max())

el k optimo es:  46  con una tasa de aciertos:  0.9333333333333333
